In [1]:
import os
import timeit

corpus_path="/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/Sentiment_Model/Oct_cust_text"
#corpus_path="/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/Tuning/Payment/pay_text_data"
%time filenames = sorted([os.path.join(corpus_path, fn) for fn in os.listdir(corpus_path)])

CPU times: user 1.92 s, sys: 222 ms, total: 2.14 s
Wall time: 2.14 s


In [2]:
len(filenames)

1142256

In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# nltk for lemmatization
from nltk.stem.wordnet import WordNetLemmatizer

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim 
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', filename='/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/Log_File.log',level=logging.DEBUG)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [10]:
import re
#noise = open("/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/Noise_words.txt").readlines()
f = open('/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/June/stop_words_upd.txt', 'r')
stop = f.read()
cus_stopwords=[i.strip() for i in stop.split(',')]
f.close()

# Remove distracting single quotes
%time  cus_stopwords= [re.sub("\'", "", sent) for sent in cus_stopwords]
# time taken : 15 min


CPU times: user 2 ms, sys: 0 ns, total: 2 ms
Wall time: 1.94 ms


In [11]:
cus_stopwords=frozenset(cus_stopwords)

In [12]:
import re
#noise = open("/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/Noise_words.txt").readlines()
f = open('/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/Sentiment_Model/Senti_noise_words.txt', 'r')
noise = f.read()
my_stopwords=[i.strip() for i in noise.split(',')]
f.close()

# Remove distracting single quotes
%time  my_stopwords= [re.sub("\'", "", sent) for sent in my_stopwords]
# time taken : 15 min

CPU times: user 41 ms, sys: 999 µs, total: 42 ms
Wall time: 39.3 ms


In [13]:
cus_stopwords=cus_stopwords.union(my_stopwords) 

In [17]:
all_data = [y for x in data_words for y in x]

from collections import Counter

freq_data=Counter(all_data)

In [18]:
l=[]
for k,v in freq_data.items():
    if v<100:
        l.append(k)

In [19]:
print len(l)

28888


In [20]:
cus_stopwords=cus_stopwords.union(l) 

In [21]:
len(cus_stopwords)

46850

In [46]:
# load data
import csv
with open('/data1/call_miner/Vinyas_Call_miner/call_miner/Pallavi/October_2018/data_words.csv','rU') as f1:
    data_words=list( csv.reader(f1) )

In [22]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in cus_stopwords] for doc in texts]

In [23]:
# Remove Stop Words
%time data_words_nostops = remove_stopwords(data_words)

# save data
import csv
with open('/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/Sentiment_Model/data_words_nostops.csv', 'w') as f1:
    writefile = csv.writer(f1)
    writefile.writerows(data_words_nostops)

CPU times: user 36min 54s, sys: 12.1 s, total: 37min 6s
Wall time: 37min 2s


In [2]:
# load data
import csv
with open('/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/Sentiment_Model/data_words_nostops.csv','rU') as f1:
    data_words_nostops=list( csv.reader(f1) )

In [3]:
# Lemmatize the documents.
import pattern.en as en

%time data_lemmatized = [[en.lemma(token) for token in doc] for doc in data_words_nostops]

CPU times: user 11min 46s, sys: 7.66 s, total: 11min 54s
Wall time: 11min 53s


In [13]:
with open("/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/Sentiment_Model/data_words_nostops_freq.csv",'w') as f:
    for k,v in  freq_data.items():
        f.write( "{} {}\n".format(k,v) )

In [47]:
print data_words[0]

['if', 'you', 'havent', 'already', 'you', 'can', 'start', 'hello', 'services', 'my', 'name', 'is', 'andrew', 'can', 'help', 'you', 'today', 'yes', 'can', 'you', 'tell', 'me', 'how', 'many', 'points', 'have', 'and', 'when', 'thanks', 'bye', 'the', 'points', 'mom', 'or', 'the', 'dollar', 'toward', 'dollars', 'im', 'sorry', 'may', 'please', 'verify', 'your', 'name', 'margaret', 'morrison', 'thank', 'you', 'so', 'much', 'for', 'that', 'one', 'im', 'just', 'hoping', 'few', 'words', 'just', 'with', 'the', 'family', 'bye', 'ritual', 'upon', 'checking', 'your', 'memory', 'we', 'have', 'four', 'certificates', 'for', 'them', 'that', 'is', 'still', 'valid', 'okay', 'in', 'my', 'account', 'okay', 'the', 'other', 'one', 'is', 'expiring', 'today', 'this', 'is', 'thirty', 'five', 'dollar', 'reward', 'and', 'then', 'it', 'expires', 'directly', 'today', 'at', 'one', 'eight', 'six', 'waiting', 'to', 'be', 'fifty', 'five', 'dollars', 'okay', 'for', 'november', 'theres', 'ten', 'dollar', 'reward', 'for', 

In [48]:
print data_words_nostops[0]

['havent', 'services', 'help', 'points', 'points', 'sorry', 'verify', 'checking', 'valid', 'account', 'expiring', 'reward', 'expires', 'reward', 'reward', 'reward', 'last', 'day', 'like', 'correct', 'appreciate', 'business', 'care']


In [3]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [5]:
#Add domain/use-case based lexicon
new_words={
             'right' : 2,'correct' : 3,'satisfied' : 3,'charge' : -2,'off' : -2,'fee' : -2,'problem' : -3,'appreciate' : 3,'definitely' : 4,'fine' : 2,'doesn' : -3,'never' : -4,'different' : -2,'apologize' : -1,'satisfaction' : 2,'wasn' : -1,'waiting' : -2,'nothing' : -3,'always' : 3,'charged' : -3,'little' : 1,'least' : -1,'happy' : 2,'most' : 2,'exactly' : 1,'wonderful' : 3,'cancel' : -3,'reset' : -2,'perfect' : 3,'slightly' : 1,'concerns' : -3,'fees' : -1,'close' : -3,'wrong' : -3,'fraud' : -2,'low' : -2,'inconvenience' : -3,'worry' : -3,'issue' : -2,'unfortunately' : -3,'kinda' : -1,'couldn' : -2,'concern' : -2,'glad' : 2,'closed' : -3,'nice' : 2,'easy' : 2,'lower' : -1,'stop' : -1,'shouldn' : -2,'cannot' : -3,'trouble' : -4,'hard' : -2,'bad' : -2,'cut' : -2,'recommend' : 3,'block' : -3,'dispute' : -4,'feedback' : -2,'missing' : -3,'forgot' : -1,'pleasure' : 3,'cancelled' : -3,'big' : 3,'longer' : -4,'isn' : -2,'improvement' : -3,'problems' : -4,'supervisor' : -3,'enjoy' : 2,'improve' : -2,'helps' : 2,'helping' : 2,'changes' : -2,'successfully' : 3,'appreciated' : 4,'repeat' : -2,'notice' : -2,'cool' : 2,'reported' : -1,'missed' : -2,'declined' : -3,'invalid' : -3,'approved' : 2,'match' : 2,'delinquent' : -2,'issues' : -3,'confused' : -2,'worries' : -3,'lose' : -2,'investigation' : -2,'nope' : -1,'temporary' : -1,'cleared' : 1,'hopefully' : 2,'wow' : 4,'crazy' : -3,'specialist' : -3,'courtesy' : 1,'locked' : -3,'charging' : -3,'stolen' : -2,'concerned' : -3,'shut' : -2,'large' : 3,'love' : 3,'decline' : -3,'wonder' : 2,'replacement' : -1,'returned' : -1,'obviously' : 3,'possibly' : 2,'delay' : -3,'keeps' : -3,'helpful' : 3,'several' : 3,'correctly' : 3,'forget' : -2,'expired' : -3,'interested' : 2,'switch' : -3,'recognize' : -3,'anymore' : -3,'expire' : -3,'pardon' : -3,'policy' : -3,'worried' : -4,'alert' : -3,'fraudulent' : -3,'technical' : -4,'mistake' : -3,'error' : -3,'drop' : -3,'confusion' : -3,'somehow' : 1,'easier' : 2,'ain' : -2,'weird' : -2,'modify' : 2,'finally' : -2,'quickly' : 2,'incorrect' : -3,'canceled' : -3,'older' : -2,'interesting' : 3,'higher' : 3,'aren' : -2,'shit' : -3,'safe' : 3,'dont' : -2,'recurring' : -2,'afraid' : -2,'chase' : -2,'honestly' : 2,'ended' : -2,'break' : -2,'excellent' : 4,'loss' : -2,'often' : -3,'closing' : -3,'sick' : -2,'extremely' : -3,'restrictions' : -3,'disputed' : -3,'following' : -3,'funny' : -2,'sold' : -2,'refunded' : -1,'unable' : -3,'force' : -2,'awesome' : 4,'negative' : -3,'fast' : 2,'honest' : 3,'kidding' : -3,'disconnected' : -3,'occurring' : -2,'fault' : -3,'expecting' : -2,'loyalty' : 2,'gosh' : -1,'lock' : -2,'corrected' : -1,'curious' : -1,'stuck' : -2,'hadn' : -2,'confusing' : -2,'kill' : -1,'rid' : -1,'acceptable' : 2,'major' : -2,'agree' : 2,'ridiculous' : -4,'trust' : 3,'war' : -3,'goodness' : 3,'notify' : -1,'congratulations' : 3,'worse' : -2,'restore' : -1,'denied' : -3,'against' : -2,'upset' : -3,'reminder' : -1,'resolve' : 2,'multiple' : -3,'expires' : -3,'compromise' : -3,'immediate' : 2,'strange' : -1,'attempted' : 1,'messed' : -3,'accepted' : 2,'successful' : 3,'blessed' : 4,'difficult' : -2,'super' : 2,'alone' : -2,'doubt' : -2,'pain' : -3,'disputing' : -3,'filed' : -2,'cancellation' : -3,'screwed' : -4,'difficulties' : -2,'hate' : -4,'stopped' : -2,'expected' : -1,'difficulty' : -2,'attorney' : -4,'investigate' : -2,'decrease' : -2,'damage' : -3,'plenty' : 2,'perfectly' : 4,'mostly' : 2,'accident' : -2,'serious' : -3,'lovely' : 3,'properly' : 2,'dropped' : -2,'collections' : -3,'frustrating' : -4,'willing' : 2,'disconnect' : -2,'canceling' : -2,'affected' : -3,'advantage' : 2,'limited' : -1,'deactivated' : -2,'barely' : -2,'mess' : -3,'restriction' : -3,'straightened' : -2,'crap' : -4,'throw' : -2,'sudden' : -2,'reduce' : -2,'sensitive' : -2,'emergency' : -3,'mystery' : -3,'bother' : -3,'unlock' : -2,'stupid' : -4,'klein' : -3,'satisfy' : 3,'lucky' : 3,'discard' : -1,'blocked' : -3,'fantastic' : 4,'odd' : -2,'investigations' : -2,'waste' : -3,'poor' : -3,'misplaced' : -2,'damn' : -2,'mad' : -3,'clearly' : 3,'delayed' : -3,'benefit' : 2,'worst' : -4,'wondered' : -2,'concerning' : -2,'hardly' : -3,'positive' : 3,'stress' : -3,'losing' : -2,'deleted' : -2,'incorrectly' : -3,'accidentally' : -1,'greatly' : 4,'accurate' : 3,'huge' : 3,'frustrated' : -4,'helped' : 3,'compromised' : -3,'terrible' : -4,'irate' : -4,'forever' : 2,'surprised' : -2,'sue' : -4,'friendly' : 3,'bless' : 3,'risk' : -2,'killed' : -2,'hell' : -4,'expedite' : -3,'dark' : -2,'pleasant' : 3,'comfortable' : 3,'destroy' : -3,'reduced' : -2,'larger' : 2,'fucking' : -4,'misses' : -2,'approximately' : 1,'favorite' : 3,'expiring' : -2,'understatement' : -2,'impressed' : 3,'disputes' : -3,'suspicious' : -2,'scared' : -3,'broken' : -2,'expensive' : -2,'complaint' : -4,'theft' : -3,'investigated' : -3,'insufficient' : -3,'duplicate' : -2,'penalty' : -3,'screw' : -4,'impact' : -2,'fairly' : 1,'glitch' : -3,'heck' : -2,'truly' : 2,'mental' : -2,'killing' : -2,'freaking' : -3,'declining' : -3,'refuse' : -3,'awful' : -4,'decent' : 2,'blocking' : -3,'trigger' : -1,'surely' : 3,'freeze' : -3,'affecting' : -2,'temporarily' : -1,'quit' : -3,'quicker' : 2,'warn' : -2,'strictly' : -2,'pushed' : -2,'nervous' : -3,'unusual' : -3,'straighten' : -2,'biggest' : 3,'triggered' : -2,'sufficient' : 2,'overdue' : -2,'attempting' : -2,'pressure' : -2,'sad' : -3,'cheap' : -2,'spam' : -2,'crying' : -2,'misunderstood' : -2,'mystic' : -2,'declines' : -3,'hurt' : -2,'fear' : -3,'blocks' : -3,'restricted' : -2,'failed' : -3,'messing' : -4,'everytime' : -3,'careful' : -2,'escalated' : -4,'tough' : -3,'affects' : -2,'horrible' : -4,'constantly' : -3,'mistaken' : -2,'seriously' : -2,'minor' : -1,'unfortunate' : -2,'differently' : -2,'tiny' : -1,'complaints' : -4,'notices' : -1,'frozen' : -2,'threw' : -2,'fuck' : -4,'surprise' : 2,'privacy' : 2,'kicked' : -4,'freezing' : -3,'discrepancy' : -3,'constant' : 2,'breaks' : -3,'wars' : -4,'tricky' : -3,'rude' : -4,'leak' : -2,'reactivate' : -2,'potentially' : -2,'knocked' : -2,'regularly' : -2,'pops' : -1,'permanently' : -2,'highest' : 4,'complicated' : -4,'detected' : -1,'damaged' : -3,'complex' : -3,'suddenly' : -1,'extreme' : 3,'escalation' : -4,'disappear' : -3,'legally' : -4,'attempts' : -3,'thrown' : -3,'dangerous' : -4,'refreshing' : 3,'potential' : -2,'popping' : -2,'critical' : -4,'reject' : -3,'negatively' : -3,'frustration' : -4,'shocked' : -4,'destroyed' : -4,'warning' : -3,'punch' : -3,'nah' : -3,'mentally' : -2,'badly' : -4,'inconveniences' : -4,'hurry' : -2,'harder' : -3,'exciting' : 3,'dropping' : -2,'awfully' : -3,'withdraw' : -2,'suspected' : -2,'joke' : -3,'policies' : -3,'periodically' : 2,'sucks' : -4,'penalized' : -4,'scam' : -3,'terribly' : -4,'impossible' : -3,'carefully' : -2,'silly' : -4,'rare' : -2,'locking' : -2,'investigating' : -2,'drops' : -2,'lowered' : -3,'excited' : 3,'interrupted' : -3,'success' : 3,'urgent' : -2,'shock' : -3,'disabled' : -3,'disappears' : -3,'mistakes' : -3,'froze' : -3,'bug' : -3,'scare' : -4,'misinformation' : -4,'confuse' : -3,'screaming' : -3,'downgrade' : -4,'complaining' : -4,'bothered' : -3,'screws' : -4,'penalties' : -4,'torture' : -4,'escalate' : -4,'conflict' : -3,'triggers' : -3,'strongly' : 3,'nasty' : -4,'annoying' : -3,'screwing' : -4,'kicking' : -3,'freak' : -3,'breaches' : -3,'bounced' : -3,'alarm' : -3,'leaking' : -3,'interrupting' : -3,'hurting' : -3,'kline' : -2,'wasted' : -3,'mysteries' : -2,'inactive' : -2,'terrific' : 4,'stealing' : -2,'efficient' : 3,'dumb' : -3,'bastard' : -4,'angry' : -3,'wasting' : -3,'evil' : -3,'victim' : -3,'threat' : -4,'recommendation' : 3,'embarrassing' : -4,'struggling' : -3,'inactivity' : -2,'dirty' : -3,'wonderfully' : 4,'wins' : 2,'victimized' : -3,'shocking' : -3,'refused' : -3,'panic' : -3,'overcharged' : -4,'outrageous' : -3,'offense' : -3,'nightmare' : -4,'insane' : -3,'gentle' : 3,'worthiness' : 3,'stressful' : -3,'excessive' : 3,'encountered' : -3,'dizzy' : -2,'painful' : -3,'knocking' : -2,'impacting' : -2,'errors' : -3,'unbelievable' : -3,'freezes' : -3,'argue' : -2,'suffered' : -3,'shorter' : -2,'sealed' : -2,'safely' : 3,'quickest' : 3,'numerous' : -3,'disappointed' : -3,'apology' : -3,'unhappy' : -4,'satisfying' : 4,'ripped' : -3,'questionable' : -3,'pissed' : -4,'penalize' : -4,'longest' : -3,'limiting' : -2,'honesty' : 3,'hardest' : -4,'advocate' : -3,'struggle' : -3,'refusing' : -3,'pride' : 3,'misstatements' : -3,'miserable' : -4,'massacre' : -3,'fraudulently' : -3,'embarrassed' : -4,'dumbest' : -4,'ban' : -3,'troubles' : -4,'interruption' : -3,'illegal' : -4,'happily' : 3,'gently' : 3,'faith' : 3,'crisis' : -3,'continuous' : 3,'compliment' : 3,'yelling' : -3,'worrying' : -2,'trusting' : 3,'stressed' : -3,'softly' : 2,'scream' : -3,'probable' : 2,'lowering' : -2,'happier' : 3,'greedy' : -2,'fussing' : -3,'forfeited' : -2,'fake' : -3,'fabulous' : 4,'courtroom' : -3,'bored' : -2,'blowing' : -2,'unexpected' : -3,'ugly' : -3,'suffering' : -3,'incredible' : 4,'freaky' : -3,'derogatory' : -3,'defect' : -3,'deceiving' : -3,'blown' : -3,'upsetting' : -4,'misery' : -3,'kicks' : -3,'exceptional' : 4,'discontinued' : -2,'critically' : -3,'acquitted' : 1,'accurately' : 3,'violation' : -4,'tremendous' : 4,'prevents' : -2,'mistakenly' : -2,'messy' : -3,'idiot' : -3,'hyper' : -3,'decreases' : -3,'craze' : -3,'bullshit' : -4,'bitch' : -4,'uncomfortable' : -3,'smiling' : 3,'ruin' : -3,'punished' : -3,'preventing' : -3,'nonsense' : -4,'massacred' : -3,'intentionally' : -3,'hiding' : -3,'hazard' : -4,'grateful' : 3,'escalations' : -4,'disagree' : -2,'buckle' : -2,'boldly' : 2,'attacked' : -3,'trauma' : -4,'mammoth' : -3,'freaked' : -3,'defective' : -4,'consistently' : 3,'occasionally' : -1,'hopeful' : 2,'escalating' : -4,'enjoying' : 3,'downgraded' : -3,'doubts' : -2,'discarded' : -2,'deadly' : -3,'absurd' : -2,'absolute' : 3,'terror' : -3,'significantly' : 3,'repeated' : -3,'rejection' : -3,'pushy' : -3,'misunderstand' : -3,'committed' : 4,'cheapest' : -2,'abuse' : -4,
    'unsuccessful' : -4,'triggering' : -3,'toxic' : -4,'termination' : -3,'terminated' : -3,'surprises' : 3,'sincerely' : 3,'prevented' : 2,'mightily' : 3,'marvelous' : 4,'functional' : 2,'flawless' : 4,'denial' : -2,'crappy' : -3,'arguing' : -2,'suspicion' : -2,'surprising' : 2,'smallest' : -1,'protest' : -2,'followup' : -2,'embarrass' : -4,'drastically' : -3,'discrepancies' : -3,'aggravating' : -3,'tremendously' : -3,'sucking' : -4,'quicken' : 2,'offending' : -2,'lawyers' : -3,'discourage' : -2,'deficient' : -2,'compromising' : -2,'bizarre' : -2,'pleasures' : 3,'offensive' : -3,'odds' : -2,'negatives' : -2,'misleading' : -2,'jeopardy' : -2,'illegally' : -3,'glitches' : -3,'failure' : -3,'cheated' : -4,'brighter' : 3,'yucky' : -3,'thankful' : 2,'terrified' : -2,'shuts' : -2,'satisfactory' : 3,'prohibited' : -2,'problematic' : -3,'pressured' : -3,'mysterious' : -3,'miserably' : -3,'dissatisfied' : -4,'disappoints' : -3,'destroying' : -3,'desperation' : -3,'depressed' : -4,'deny' : -3,'decree' : -3,'crashed' : -3,'complained' : -4,'wrecked' : -3,'warned' : -3,'mistreat' : -4,'helpless' : -3,'harsh' : -3,'handicapped' : -3,'guilt' : -3,'frustrate' : -4,'fluctuate' : -3,'depression' : -4,'deceptive' : -2,'deceit' : -3,'blurred' : -3,'weirdest' : -4,'tortured' : -4,'threatening' : -4,'tedious' : -3,'sued' : -4,'restricting' : -3,'restless' : -2,'refuses' : -2,'misspelled' : -2,'massive' : -3,'investigates' : -2,'impatient' : -2,'hardy' : -2,'happiness' : 3,'frustrates' : -4,'freaks' : -3,'disturbing' : -3,'distrust' : -3,'destruction' : -3,'deceived' : -3,'decay' : -2,'confront' : -3,'coaxed' : -3,'assault' : -3,'aggression' : -3,'aggravated' : -3,'suspiciously' : -3,'panicked' : -3,'lol' : 3,'insult' : -3,'incomplete' : -3,'humiliated' : -4,'fucked' : -4,'disturb' : -3,'desperately' : -3,'depriving' : -3,'depressing' : -3,'charming' : 3,'bloody' : -3,'attraction' : 3,'aggressive' : -4,'worthless' : -4,'worsen' : -2,'useless' : -4,'unlikely' : -2,'unfair' : -4,'threats' : -4,'thanked' : 3,'tempting' : 3,'tempers' : 3,'sucked' : -3,'stuffy' : -3,'stronger' : 3,'punish' : -3,'praise' : 3,'policing' : -3,'misspelling' : -2,'misinformed' : -3,'irritated' : -4,'hurts' : -3,'frowned' : -3,'drastic' : -3,'disqualify' : -2,'anxious' : -2,'amusing' : 3,'unusually' : -2,'terrorism' : -3,'resistance' : -2,'punishing' : -3,'pressuring' : -3,'pisses' : -3,'panicking' : -4,'offended' : -2,'harassing' : -3,'frustrations' : -4,'embarrassment' : -4,'efficiently' : 4,'easing' : 3,'ditch' : -2,'dishonest' : -3,'disapprove' : -2,'disappointing' : -4,'disappoint' : -4,'detriment' : -3,'denying' : -3,'bluntly' : -3,'alleged' : -2,'unsuccessfully' : -4,'truthfully' : 3,'troubling' : -3,'troubled' : -4,'traitor' : -3,'stunned' : -2,'redundant' : -2,'recommendations' : 3,'mocking' : -2,'mock' : -2,'misusing' : -2,'mistreatment' : -3,'insulting' : -4,'improperly' : -3,'hated' : -4,'harmful' : -3,'downgrading' : -3,'disturbed' : -3,'disapproved' : -2,'crazier' : -3,'cheating' : -3,'challenging' : -3,'annoyed' : -4,'violated' : -4,'upsets' : -4,'uncanny' : -4,'threaten' : -4,'thankfully' : 4,'stupidity' : -3,'stubborn' : -3,'strangest' : -3,'screams' : -3,'reluctant' : -2,'punishment' : -3,'painfully' : -3,'overwhelming' : 3,'oppression' : -2,'opposing' : -3,'nastier' : -3,'incompetent' : -3,'hassles' : -2,'harassed' : -4,'happiest' : 4,'fooling' : -3,'fluctuates' : -2,'disqualified' : -2,'disinterested' : -2,'disconnects' : -2,'discomfort' : -2,'cranky' : -3,'confronted' : -3,'conflicting' : -3,'competent' : 3,'clueless' : -2,'assaulting' : -2,'assassinate' : -3,'apologizing' : -2,'aggravate' : -3,'abused' : -4,'violent' : -4,'stumbled' : -2,'struggles' : -3,'puzzles' : -3,'puzzled' : -3,'provoke' : -4,'optimistically' : 4,'notorious' : -3,'neglected' : -2,'nastiness' : -3,'misspell' : -2,'mismatched' : -2,'manipulate' : -2,'irritating' : -3,'ironically' : -2,'insults' : -3,'horror' : -3,'hopeless' : -4,'hating' : -4,'excessively' : -3,'escalates' : -4,'disrespect' : -3,'disgusting' : -4,'discriminate' : -4,'discontent' : -3,'disappointment' : -4,'disappearing' : -2,'disagrees' : -2,'disagreeing' : -3,'disadvantage' : -3,'devastation' : -3,'destructive' : -3,'destruct' : -3,'craziness' : -3,'conceal' : -2,'awkward' : -3,'appreciating' : 3,'adversely' : -3,'accused' : -3,'worthy' : 3,'worrisome' : -2,'unstable' : -2,'unpredictable' : -2,'unfriendly' : -3,'threatened' : -4,'terrifying' : -4,'terminating' : -3,'stupidest' : -4,'seizure' : -2,'seizing' : -2,'saddened' : -3,'rudeness' : -4,'rotten' : -2,'resistant' : -2,'outrageously' : -2,'outraged' : -3,'offender' : -3,'nonsensical' : -3,'negativity' : -3,'intimidate' : -2,'inefficient' : -2,'inappropriately' : -2,'inaccurate' : -2,'foolish' : -2,'doubtful' : -2,'displeased' : -2,'disastrous' : -4,'disagreed' : -2,'disabling' : -2,'dirtier' : -2,'devastate' : -3,'denies' : -3,'denials' : -3,'complicates' : -3,'complains' : -4,'blockage' : -3,'bitter' : -3,'awaited' : -2,'applauded' : 3,'applaud' : 3,'aches' : -2,'abrupt' : -2,'yelled' : -3,'walkout' : -2,'vandalized' : -3,'unused' : -3,'unseen' : -1,'unreliable' : -3,'unlike' : -2,'unfamiliar' : -2,'traps' : -3,'trapped' : -3,'tragically' : -4,'tougher' : -2,'suspicions' : -2,'suicidal' : -3,'sighs' : -3,'sighed' : -3,'shatter' : -3,'shameless' : -4,'severely' : -4,'sentimental' : -2,'selfish' : -3,'seizures' : -2,'scratchy' : -1,'sadistic' : -3,'ruining' : -3,'rudely' : -4,'restricts' : -3,'restrictive' : -3,'resisted' : -2,'regrettably' : 2,'proceedings' : -2,'precarious' : -3,'pointless' : -3,'piss' : -3,'misuse' : -3,'misquoted' : -4,'mismanagement' : -4,'mismanaged' : -4,'misfits' : -3,'misconstrued' : -3,'likable' : 3,'lamenting' : -3,'joyful' : 3,'interruptions' : -3,'intensified' : -3,'instigated' : -3,'insanity' : -3,'inconveniencing' : -4,'inconsistency' : -3,'immensely' : 3,'ignorance' : -2,'humongous' : -2,'hostile' : -3,'horrors' : -3,'horrified' : -4,'horrendous' : -3,'hinders' : -2,'hiccups' : -2,'hesitation' : -2,'goofed' : -2,'fluctuating' : -2,'flickering' : -2,'extraordinary' : 3,'disturbance' : -2,'distractions' : -2,'distort' : -2,'disgusted' : -3,'disasters' : -2,'compromises' : -2,'compelling' : -2,'brutally' : -4,'brutal' : -4,'brilliant' : 4,'brightest' : 4,'bothersome' : -3,'atrocity' : -4,'ashamed' : -3,'arrogance' : -2,'arrested' : -2,'apologetic' : -2,'yells' : -3,'wrongly' : -2,'wrongdoing' : -2,'wastes' : -2,'wasteful' : -2,'vulgar' : -3,'voracious' : -3,'violations' : -3,'unprofessional' : -4,'unpleasant' : -3,'unexplained' : -2,'unexpectedly' : -2,'uneasy' : -2,'unduly' : -2,'troublesome' : -3,'torturing' : -3,'terrors' : -3,'terrifies' : -3,'superbly' : 3,'superb' : 4,'shameful' : -3,'severed' : -3,'screamed' : -3,'sarcastic' : -3,'sarcasm' : -3,'ruthless' : -3,'ridiculously' : -4,'prosecutors' : -3,'prosecutor' : -4,'profanity' : -4,'pissing' : -4,'overwhelmed' : 3,'overreact' : -3,'outrage' : -3,'offensively' : -3,'oddly' : -3,'negate' : -2,'monstrous' : -2,'molested' : -3,'mislead' : -3,'mislabeled' : -3,'misinterpreted' : -2,'misfortune' : -2,'misconduct' : -3,'marvellous' : 4,'manipulated' : -3,'leaked' : -2,'largely' : 3,'lacks' : -3,'irritates' : -4,'irregular' : -3,'irony' : -3,'ironic' : -3,'insanely' : -3,'inconvenienced' : -3,'inconsistent' : -4,'impatience' : -2,'illegals' : -3,'hyped' : -3,'hurtful' : -3,'hurdle' : -2,'humiliating' : -4,'huddled' : -2,'huddle' : -2,'horrific' : -3,'horribly' : -4,'hesitant' : -1,'helplessly' : -2,'haunted' : -3,'hatred' : -3,'harshly' : -3,'harass' : -4,'growling' : -3,'gracefully' : 3,'graceful' : 3,'ghastly' : -3,'fruitless' : -3,'fruitful' : 3,'explosive' : -3,'exploiting' : -4,'exploding' : -3,'expensively' : -3,'expel' : -3,'dysfunction' : -3,'dramatically' : -3,'distracting' : -2,'dissatisfaction' : -4,'disruption' : -3,'disrupting' : -3,'disrespectful' : -3,'displeasure' : -2,'dirtiest' : -4,'devastating' : -4,'devalued' : -2,'detrimental' : -3,'deteriorates' : -3,'destructing' : -3,'destroyer' : -3,'dangerously' : -4,'damaging' : -4,'crashes' : -3,'corrupting' : -3,'corrupted' : -3,'contradicts' : -2,'conspiracy' : -3,'choking' : -3,'ceases' : -2,'betraying' : -3,'assholes' : -3,'asshole' : -3,'apologized' : -2,'anomaly' : -2,'anguish' : -3,'amazingly' : 3,'agony' : -3,'agitated' : -4,'aggressively' : -4,'wrongfully' : -2,'worsening' : -3,'wicked' : -3,'whimsical' : -2,'weirdly' : -2,'vindicated' : -3,'unwilling' : -2,'untruthful' : -3,'untrained' : -2,'untimely' : -2,'unsure' : -2,'unsolved' : -3,'unskilled' : -3,'unsettled' : -3,'unsecured' : -2,'unruly' : -3,'unrestricted' : -2,'unreasonable' : -2,'unrealized' : -1,'unraveling' : -2,'unravel' : -2,'unprotected' : -2,'unnerving' : -3,'unnecessary' : -2,'unnecessarily' : -2,'unlawful' : -3,'unknowingly' : -2,'unjustified' : -4,'undoubtedly' : -2,'undetected' : -1,'uncertain' : -1,'unbalanced' : -1,'unattended' : -1,'toughest' : -3,'tolerated' : 3,'tireless' : -3,'threatens' : -4,'thrasher' : -4,'terrorized' : -4,'terrorize' : -4,'terrifically' : 3,'tampering' : 4,'tampered' : -3,'superfluous' : -2,'superficial' : -2,'sues' : -4,'speechless' : -2,'slashing' : -2,'slashed' : -2,'slapping' : -4,'skillfully' : 3,'sensational' : 3,'sacrifices' : -2,'rigorous' : -2,'rightfully' : 3,'rightful' : 3,'refuted' : -2,'refusal' : -2,'redundancy' : -2,'redefine' : -1,'rebuke' : -3,'rascals' : -3,'punishments' : -4,'provoking' : -4,'provoked' : -4,'provocation' : -4,'profusely' : -3,'powerfully' : 3,'popularity' : 3,'poorest' : -3,'poisoning' : -3,'plentiful' : 3,'pleasantly' : 3,'perplexed' : -3,'perfectionist' : 4,'penalizes' : -4,'paralyzing' : -3,'oppressive' : -3,'opponent' : -3,'omitted' : -3,'offensives' : -3,'offenses' : -3,'obstructed' : -3,'nothin' : -2,'nosy' : -2,'neglecting' : -2,'neglect' : -2,'needlessly' : -2,'mysteriously' : -3,'motivates' : 3,'molesting' : -3,'mocked' : -3,'mistreating' : -3,'mishandling' : -2,'misconception' : -2,'mischief' : -2,'miscalculation' : -2,'miraculous' : 3,'mindlessly' : -3,
    'mindless' : -3,'mediocre' : -2,'massively' : -3,'maligned' : -2,'malicious' : -3,'malfunction' : -3,'litigate' : -4,'lawsuits' : -3,'lamented' : -2,'lament' : -2,'knocks' : -2,'knockout' : -2,'jobless' : -1,'jittery' : -2,'jeopardizing' : -2,'irritation' : -3,'irritate' : -4,'irrelevant' : -2,'irregularities' : -2,'intolerant' : -3,'intolerance' : -3,'intimidation' : -2,'interrupts' : -2,'interrogation' : -2,'interrogate' : -2,'intermittently' : -1,'interfered' : -2,'insulted' : -2,'insulating' : -2,'insufficiency' : -2,'insensitivity' : -1,'insensitive' : -2,'inhibit' : -1,'infuriated' : -3,'infected' : -2,'illegitimate' : -3,'idiots' : -3,'humiliation' : -3,'humiliate' : -4,'hostility' : -3,'hindering' : -2,'hinder' : -2,'hiked' : -2,'hiccup' : -1,'hesitated' : -1,'hesitantly' : -1,'hazardous' : -2,'hateful' : -3,'harmed' : -3,'harassment' : -3,'grumpy' : -2,'grudgingly' : -2,'grudges' : -2,'grudge' : -2,'fucker' : -3,'frightens' : -3,'frighten' : -3,'friendliest' : 4,'foolishly' : -3,'extraordinarily' : 4,'erroneously' : -2,'distorting' : -3,'disliked' : -2,'disheartening' : -3,'disgust' : -3,'discrimination' : -4,'discontinuing' : -3,'delighted' : 3,'complicating' : -3,'compelled' : -3,'cluttered' : -3,'brightens' : 4,'brighten' : 4,'arrogant' : -4,'annoys' : -3,'annoyance' : -3,'annoy' : -3,'animated' : -3,'angers' : -3,'angered' : -3,'aggressiveness' : -3,'aggravates' : -3,'abusive' : -4,'abnormally' : -2,'abnormal' : -2
          } 

In [4]:
domain_words_scores={'account':1,'close':-4, 'payment':1, 'card':1, 'know':1, 'not':-4, 'number':1, 'dont':-2, 'like':1, 'credit':1.1, 'make':1.2, 'last':1, 'help':2, 'got':1.3, 'need':1.4, 'pay':2, 'phone':2.2, 'sorry':1.8, 'first':1, 'balance':1.7, 'check':1.5, 'day':1, 'time':1, 'bank':1.9, 'due':0.9, 'correct':2, 'good':1.6, 'address':2, 'said':1, 'late':-4, 'use':1, 'date':1.5, 'new':1, 'information':1.3, 'mail':1.6, 'month':1.9, 'statement':1.8, 'zero':0, 'fee':-3.5, 'didnt':-3, 'want':0.8, 'charge':-3.9, 'bill':1.9, 'why':-2.5, 'representative':1.1, 'great':2.2, 'care':0.3, 'hold':1.1, 'able':1, 'made':1.8, 'send':1.9, 'debit':1.3, 'paid':1, 'appreciate':3, 'checking':1.7, 'amount':1.5, 'online':1.8, 'trying':1.5, 'service':1.8, 'more':1.5, 'received':1.1, 'business':1.4, 'problem':-2.8, 'receive':1, 'available':1.8, 'interest':-1.2, 'set':1, 'cant':-3, 'verify':2, 'system':0.1, 'question':0.7, 'try':1.1, 'payments':1, 'line':1, 'store':0.4, 'sent':0, 'called':1, 'hours':1, 'rate':1.6, 'services':1, 'process':1.9, 'second':1, 'work':1.4, 'confirmation':1.5, 'whats':1.1, 'change':1.4, 'code':1.5, 'understand':1.4, 'car':0.4, 'home':0.6, 'never':-2.9, 'charges':-1.4, 'reason':0.5, 'purchase':1.3, 'used':1.4, 'minimum':1.6, 'id':1.4, 'entire':1.4, 'getting':0.5, 'money':0.6, 'guess':0.7, 'showing':0.8, 'thought':0.9, 'likely':1, 'file':1.1, 'making':1.4, 'free':0, 'close':0.5, 'security':0.5, 'master':0.7, 'using':0.1, 'automated':1.2, 'current':1.2, 'wait':0.4, 'numbers':1.1, 'old':1.1, 'monitored':1.1, 'request':0.9, 'looking':1.1, 'find':1.2, 'transfer':1.4, 'different':1.6, 'currently':1.6, 'show':1.4, 'looks':1.3, 'told':1.2, 'letter':1.4, 'terms':1.2, 'message':1.3, 'user':1.1, 'doesnt':-1.5, 'open':1, 'order':1.2, 'transaction':1.3, 'completely':1.2, 'happened':1.3, 'score':1.4, 'update':1.5, 'complete':1.6, 'social':1.4, 'past':1.4, 'billing':1.2, 'wont':-0.5, 'expiration':0.3, 'mailing':0.5, 'authorize':1.3, 'debt':0.5, 'fees':-1.4, 'havent':-3.4, 'promotion':1.3, 'happy':1.9, 'password':1.1, 'confirm':1.1, 'telephone':1.2, 'needed':1.1, 'posted':1.1, 'log':0.9, 'cards':1.1, 'updated':1.2, 'routing':1.4, 'lost':-0.6, 'post':1, 'text':1.1, 'paying':1.2, 'charged':-2.9, 'electronic':1, 'remove':1.2, 'pull':1.1, 'website':1.3, 'high':1.2, 'leave':1.2, 'paper':1.3, 'modify':1.2, 'best':1.3, 'person':1.4, 'read':0.5, 'access':1.3, 'place':1.3, 'wrong':-1.5, 'cancel':1.4, 'add':1.4, 'authorized':1.5, 'fraud':-3, 'missed':-1.4, 'statements':-1.5, 'limit':1.4, 'closed':1.5, 'rating':1.6, 'cannot':-2.6, 'member':1, 'issue':-4, 'bills':1.2, 'customers':1.1, 'percent':0.5, 'low':-1.1, 'wasnt':-1.4, 'accounts':1.3, 'double':1.3, 'miss':-1.4, 'cd':1.3, 'rewards':1.7, 'activate':1.9, 'write':1.5, 'removed':-4.5, 'activated':1.4, 'plan':1.1, 'membership':1.1, 'completed':1.1, 'auto':1.3, 'receiving':1.4, 'confirms':1.1, 'promotional':1.5, 'transactions':1.4, 'birth':1.4, 'future':1.1, 'without':0.9, 'computer':1.1, 'visa':1.1, 'additional':1.2, 'zip':1.2, 'street':1.2, 'option':1.3, 'register':1.4, 'increase':1.1, 'unfortunately':-0.5, 'automatically':-1.1, 'mobile':1.1, 'income':1.1, 'applied':1.2, 'city':1.2, 'couldnt':1.5, 'contacted':1.5, 'wouldnt':-1.2, 'details':1, 'cycle':1, 'reset':1.2, 'wave':-0.5, 'dispute':-1.2, 'processed':0.9, 'po':1.2, 'apartment':1.3, 'waive':-0.8, 'mailed':1.2, 'apply':1.2, 'manager':0.6, 'recommend':1.3, 'approved':1.1, 'application':1.3, 'improvement':1.3, 'transferred':1.4, 'courtesy':1.9, 'verification':1.4, 'schedule':1.4, 'stop':0.9, 'key':0.8, 'returned':1.2, 'offer':1.4, 'submit':1.5, 'valid':1.3, 'canceled':1.5, 'travel':1.6, 'chance':1.6, 'primary':1.2, 'declined':-2.7, 'refund':1.4, 'pending':-1.1, 'sign':1.5, 'sick':1.1, 'form':1.1, 'funds':1.3, 'successfully':1.4, 'status':1.5, 'receipt':1.1, 'activity':1.2, 'trouble':-4, 'hang':-1.1, 'recent':1.2, 'connected':1.4, 'missing':-4, 'reflect':1.4, 'processing':1.4, 'submitted':1.3, 'stolen':-1.7, 'issues':-4, 'fax':1.4, 'screen':1, 'invalid':-1.4, 'clear':1.2, 'block':-0.9, 'purchased':1.3, 'registered':1.5, 'fraudulent':-3.5, 'merchant':1.4, 'rates':1.1, 'confused':-1.1, 'waived':1.3, 'credited':1, 'options':1.3, 'unable':-1.4, 'requested':1.4, 'expired':-1.4, 'cycles':1.2, 'worries':-3.5, 'scheduled':1.4, 'discount':1.3, 'replacement':1.2, 'notice':1.2, 'moved':1.2, 'charging':-4, 'view':1.3, 'permission':1.4, 'notification':1.4, 'financial':1.5, 'delay':-4, 'promotions':1.4, 'catch':1.4, 'paperless':1.4, 'advance':1, 'hate':-2.5, 'grace':1.3, 'delete':1.3, 'item':1.2, 'waiver':-1.1, 'finance':1.1, 'temporary':1.1, 'noticed':1.1, 'personal':1.2, 'expire':1.2, 'deposit':1.3, 'adjustment':1.2, 'shouldnt':-1.2, 'closing':1.2, 'claim':1.3, 'forget':1.2, 'collections':1.2, 'deferred':1.3, 'confirming':1.3, 'signed':1.3, 'enroll':1.3, 'reporting':1.3, 'points':1.3, 'mess':-0.9, 'ordered':1.1, 'understanding':1.1, 'insurance':1.2, 'correctly':1.3, 'dates':1.4, 'initial':1.3, 'loan':1.2, 'traveling':1.3, 'confirmed':1.2, 'letters':1.1, 'approval':1.2, 'spend':-0.1, 'authorization':1.3, 'isnt':-0.9, 'locked':1.5, 'bureau':1.1, 'reward':1.4, 'enrolled':1.5, 'locate':1.1, 'expires':-1.2, 'envelope':1.2, 'sale':1.1, 'cleared':1.2, 'confusion':1.2, 'recognize':1.2, 'policy':1.4, 'balances':1.3, 'count':1.4, 'requesting':1.4, 'billed':1.1, 'incorrect':-4, 'maturity':1.3, 'interested':1.4, 'lose':-0.9, 'enrollment':1.1, 'union':1.2, 'error':-4, 'removing':1.4, 'hell':-4, 'names':1.4, 'fault':-4, 'showed':1.1, 'identity':1.3, 'weird':-2.5, 'returning':1.2, 'hadnt':-2.8, 'adjusted':1.2, 'deducted':-2.9, 'signature':1.3, 'certificate':1.4, 'located':1.5, 'appointment':1.2, 'reflected':1.4, 'logged':1.5, 'delivered':1.4, 'hurricane':1.4, 'maintenance':1.4, 'shipped':1.2, 'notify':1.3, 'expiring':1.3, 'renew':1.3, 'bureaus':1.3, 'term':1.2, 'vacation':1.3, 'attorney':1.4, 'updating':1.4, 'loss':-1.1, 'browser':1.1, 'credits':1.3, 'prefer':1.2, 'transferring':1.4, 'delivery':1.1, 'accepted':1.1, 'complaint':-3.6, 'activation':1.5, 'apr':1.4, 'technically':1.4, 'generate':1.3, 'atm':1.3, 'providers':1.4, 'offers':1.5, 'action':1.4, 'spelled':1.3, 'contacting':1.3, 'confusing':-1.1, 'recurring':1.4, 'allocated':1.5, 'messages':1.4, 'misplaced':1.3, 'design':1.4, 'owner':1.4, 'secure':1.4, 'fucking':-4, 'negative':-0.5, 'merchandise':1.1, 'sue':-4, 'resend':2, 'slow':-3.1, 'saved':1.2, 'decline':-4, 'extension':1.5, 'clarify':1.4, 'penalty':-2.5, 'disputed':-0.6, 'release':1, 'fuck':-4, 'disconnected':-4, 'cancellation':1.5, 'applicant':1.3, 'emergency':1.1, 'rejected':-3.4, 'refunded':1.4, 'transactional':1.5, 'bitch':-3.9, 'ridiculous':-4, 'technical':1.6, 'renewal':1.4, 'legal':-1.4, 'waved':1.1, 'invoice':1.3, 'cvv':1.4, 'ids':1.2, 'reconnected':1.1, 'acceptable':1.1, 'blocked':-2.8, 'liability':1.5, 'beneficiary':1.7, 'offered':1.4, 'canceling':-1.3, 'upgrade':1.8, 'investigate':1.7, 'paycheck':1.8, 'notified':1.4, 'federal':1.4, 'fedex':1.5, 'tracking':1.6, 'courtney':1.5, 'reoccurring':1.4, 'strange':-3.5, 'authorizing':1.5, 'council':1.4, 'decrease':-2.4, 'difficulty':-1.3, 'activating':1.3, 'coupon':1.4, 'submitting':1.5, 'delinquency':-3.4, 'unlocked':1.4, 'stupid':-4, 'increased':1.1, 'allocation':1.4, 'disputing':1.3, 'unauthorized':-4, 'forms':-1.1, 'upgraded':1.4, 'registration':1.4, 'compromised':-2.2, 'spanish':1.3, 'adjust':1.3, 'unlock':1.4, 'installment':1.4, 'compromise':-0.9, 'repair':-1.1, 'disconnect':-1.4, 'terrible':-3.9, 'fund':1.1, 'quit':-2.9, 'renewed':1.2, 'bankruptcy':-3.7, 'beneficiaries':1.2, 'motion':1.3, 'frustrating':-4, 'delayed':-2.5, 'qualified':1.4, 'subtract':1.3, 'frustrated':-3.9, 'damage':-2.8, 'deactivated':-3.1, 'sufficient':1.2, 'signing':1.3, 'marked':1.4, 'download':1.4, 'stuck':-2.2, 'retired':-1.3, 'suspicious':-2.5, 'worst':-2.7, 'warranty':1.5, 'reopen':1.4, 'mortgage':1.5, 'waste':-2.5, 'limits':1.3, 'alerts':1.4, 'conditions':1.5, 'retrieve':1.4, 'difficulties':-2.1, 'loading':1.5, 'disclose':1.5, 'removal':1.5, 'funded':1.4, 'prevention':1.4, 'applications':1.5, 'deduct':-1.5, 'deposits':1.4, 'declining':-3.7, 'withdrawal':1.4, 'redeem':1.3, 'deadline':1.2, 'reviewed':1.3, 'worse':-4, 'freaking':-2.6, 'digital':1.1, 'theft':-2.8, 'dogs':1.4, 'unfortunate':-2.4, 'registering':1.4, 'physically':1.1, 'ivr':1.4, 'rebate':1.4, 'government':1.5, 'yell':-3.3, 'complain':-1.1, 'qualifies':1.3, 'ordering':1.2, 'passwords':1.3, 'investigations':1.5, 'excluding':1.2, 'incurred':1.2, 'counseling':1.2, 'debited':1.4, 'authorizations':1.3, 'codes':1.2, 'failed':-4, 'discussed':1.2, 'arrived':-1.1, 'reinstated':-0.7, 'insufficient':-0.1, 'installments':1.2, 'reduction':1.1, 'texting':1.2, 'deceased':-1.5, 'awards':1.5, 'content':1.1, 'withdrawn':1.1, 'clicking':1.2, 'expectation':1.2, 'scheduling':1.3, 'misunderstood':1.4, 'military':1.4, 'coded':1.3, 'escalation':-2.4, 'coupons':1.5, 'maturing':1.7, 'condition':1.1, 'financially':1.3, 'explorer':1.1, 'rules':1.2, 'mistaken':1.1, 'ownership':1.4, 'institution':1.4, 'truth':1.4, 'traditional':1.4, 'retirement':1.5, 'rights':1.3, 'suspect':-0.2, 'rude':-3.4, 'certified':1.2, 'investigated':1.4, 'increases':1.3, 'authentication':1.4, 'destroyed':-0.7, 'notifications':1.3, 'confuse':-1.1, 'clarification':1.8, 'loyalty':2, 'recognizing':1.2, 'volume':1.2, 'secured':1, 'privacy':-2, 'technology':0.5, 'deactivate':-0.7, 'frustration':-4, 'complicated':-2.4, 'blocking':-1.8, 'overpaid':0.5, 'warning':-0.9, 'calculation':0.9, 'reimburse':-0.4, 'mismatch':-0.8, 'bullshit':-2.7, 'unemployment':1, 'investigating':1.1, 'obligation':-0.5, 'refused':-0.9, 'sucks':-2.5, 'bug':-3.7, 'troubles':-3.9, 'award':1.5, 'blocks':-1.1, 'unemployed':1.1, 'authenticate':1.1, 'unexpected':-0.9, 'suspected':-1.1, 'refunds':1, 'reactivate':1.1, 'websites':1.1, 'fraudulently':-1.2, 'waiving':1.1, 'complaints':-3.5, 'repayment':1.1, 'misleading':-4, 'harassing':-4, 'matched':0.6, 'failure':-4, 'damages':-4, 'mislead':-2.8, 'redeemable':0.7, 'resubmitted':0.7, 'inconveniences':-1.1, 'decreasing':-0.5, 'deductible':-0.7, 'misinformed':-3.8, 'escalating':-3.5, 'neglected':-0.9, 'suspecting':-0.8, 'counterfeit':-1.9, 'troubleshooting':-0.4, 'incentive':1.2, 'illegible':-0.9, 'reregister':1, 'misunderstand':0.2, 'escalations':-3.5, 'overcharged':-3.3, 'cancellations':0.8}

In [5]:
analyser.lexicon.update(domain_words_scores)

In [6]:
len(domain_words_scores)

674

In [7]:
len(analyser.lexicon)

8038

In [7]:
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))

In [19]:
type(analyser.polarity_scores("The phone is super cool."))

dict

In [21]:
sentiment_analyzer_scores("The phone is super cool.")

NameError: name 'sentiment_analyzer_scores' is not defined

In [7]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

/data1/ANACONDA/Anaconda2.7/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [7]:
len(data_lemmatized)

1132250

In [10]:
d=data_words_nostops[:5]

In [11]:
len(d)

5

In [12]:
print d[1]

['help', 'got', 'approved', 'credit', 'card', 'need', 'account', 'number', 'complete', 'purchase', 'help', 'paper', 'number', 'expiration', 'date', 'account', 'expiration', 'date', 'cvv', 'account', 'appreciate', 'bank', 'store', 'card', 'good', 'day', 'good']


In [8]:
import pandas as pd
df = pd.DataFrame([data_words_nostops])
df2=df.T

In [9]:
df2.columns=['data']

In [10]:
df2.shape

(1132250, 1)

In [109]:
print data_words_nostops[8800]

['card', 'services', 'payment', 'got', 'process', 'payment', 'leave', 'charge', 'like', 'pay', 'checking', 'know', 'process', 'payment', 'automated', 'system', 'free', 'charge', 'want', 'transfer', 'automated', 'system', 'payment']


In [108]:
analyser.polarity_scores(str(data_words_nostops[8800]))

{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}

In [126]:
b=TextBlob(str(data_words_nostops[8800]))

In [127]:
b.sentiment

Sentiment(polarity=0.4, subjectivity=0.8)

In [11]:
s=[]
data=[]
for index,row in df2.iterrows():
    #print row[0]
    s.append(analyser.polarity_scores(str(row.data)))
    #print s
    data.append(row[0])

s2=pd.DataFrame(s)    
sdf=pd.concat([pd.DataFrame(s),pd.Series(data)],axis=1)
#sdf.columns=['Score']

/data1/ANACONDA/Anaconda2.7/lib/python2.7/site-packages/vaderSentiment-3.2.1-py2.7.egg/vaderSentiment/vaderSentiment.py:252: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if token in self.emojis:


In [33]:
sdf.rename(columns={0:'Data_words'},inplace=True)

In [34]:
sdf.head(5)

,compound,neg,neu,pos,Data_words
0,0.0,0.0,1.0,0.0,"[havent, services, help, points, points, sorry..."
1,0.0,0.0,1.0,0.0,"[help, got, approved, credit, card, need, acco..."
2,0.0,0.0,1.0,0.0,"[services, mail, cards, not, deactivated, new,..."
3,0.0,0.0,1.0,0.0,"[good, bank, account, number, leave, use, card..."
4,0.0,0.0,1.0,0.0,"[first, last, card, help, need, close, account..."


In [100]:
len(sdf.loc[sdf.compound<=-0.05]) 

5

In [12]:
len(sdf.loc[sdf['compound'] >=0.05])

72

In [13]:
len(sdf.loc[sdf['compound']<=-0.05])

33

In [20]:
len(sdf[(sdf['compound']> -0.05) & (sdf['compound']<0.05)])

1132145

In [118]:
sdf.describe()

,compound,neg,neu,pos
count,1.132250e+06,1.132250e+06,1.132250e+06,1.132250e+06
mean,1.169715e-05,2.104306e-05,9.999358e-01,4.313093e-05
std,3.694671e-03,3.897776e-03,6.802349e-03,5.575043e-03
min,-3.818000e-01,0.000000e+00,2.380000e-01,0.000000e+00
25%,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
max,4.939000e-01,7.220000e-01,1.000000e+00,7.620000e-01


In [37]:
sdf.to_csv("/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/Sentiment_Model/sent_score.csv")

In [83]:
for r,i in sdf.iterrows:
    

,compound,neg,neu,pos,Data_words


In [92]:
searchTerm = 'fraud'
len(sdf[[searchTerm in x for x in sdf['Data_words']]])

60655

In [93]:
df4=sdf[[searchTerm in x for x in sdf['Data_words']]]

In [95]:
df4.head(5)

,compound,neg,neu,pos,Data_words
11,0.0,0.0,1.0,0.0,"[bank, account, online, services, problem, acc..."
25,0.0,0.0,1.0,0.0,"[entire, monitored, bank, service, account, pu..."
48,0.0,0.0,1.0,0.0,"[bank, master, last, card, number, card, numbe..."
74,0.0,0.0,1.0,0.0,"[credit, card, services, whats, missed, help, ..."
81,0.0,0.0,1.0,0.0,"[bank, help, got, text, number, phone, phone, ..."


In [102]:
df4.dtypes

compound      float64
neg           float64
neu           float64
pos           float64
Data_words     object
dtype: object

In [119]:
l2=len(df4[(df4['compound']> -0.05) & (df4['compound']<0.05)])

In [120]:
l1=len(df4.loc[df4['compound'] >=0.05])

In [118]:
l=len(df4.loc[df4['compound']<=-0.05])

In [129]:
lis=[l,l1,l2]

In [138]:
lis

[2, 4, 60649]

In [142]:
new_df=pd.DataFrame(lis).T

In [143]:
new_df

,0,1,2
0,2,4,60649


In [ ]:
new_df

In [73]:
import pandas as pd
df = pd.DataFrame(["What is the answer", 
                   "the answer isn't here, but the answer is 42" , 
                   "dogs are nice", 
                   "How are you"], columns=['words'])

In [76]:
type(df.words[0])

str

In [54]:
len(sdf.filter(like='points',axis=1))

1132250

In [98]:
t=analyser.lexicon

In [99]:
type(t)

dict

In [101]:
with open("/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/Sentiment_Model/Vader_lexicon_2.csv",'w') as f:
    for k,v in  t.items():
        f.write( "{} {}\n".format(k.encode('ascii', 'ignore'),v) )

In [27]:
l=[]
for k,v in t.items():
    l.append(k)

    

In [119]:
 from textblob import TextBlob

In [120]:
blob = TextBlob("Analytics Vidhya is a great platform to learn data science. \n It helps community through blogs, hackathons, discussions,etc.")

In [121]:
blob.sentiment

Sentiment(polarity=0.8, subjectivity=0.75)